In [173]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from joblib import dump, load
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge, Lasso
import tensorflow as tf
import warnings
from TransformationPipeline import TransformationPipeline
warnings.filterwarnings('ignore')
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error

In [174]:
data = pd.read_csv("BellouchiFinal.csv")

In [175]:
data = data.drop(columns=["Titre"])

In [176]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Prix          1128 non-null   int64 
 1   Marque        1128 non-null   object
 2   Modele        1128 non-null   object
 3   Transmission  1128 non-null   object
 4   Carburant     1128 non-null   object
 5   Annee         1128 non-null   int64 
 6   Kilometrage   1128 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 61.8+ KB


In [177]:
# features = data[['Titre', 'Marque', 'Modele', 'Transmission', 'Carburant', 'Annee', 'Kilometrage']]
# target = data['Prix']

# # Preprocessing pipeline for the description
# text_transformer = Pipeline(steps=[
#     ('tfidf', TfidfVectorizer(max_features=1000))
# ])

# # Preprocessing pipeline for categorical and numerical data
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('titre', text_transformer, 'Titre'),
#         ('cat', OneHotEncoder(handle_unknown='ignore'), ['Marque', 'Modele', 'Transmission', 'Carburant']),
#         ('num', StandardScaler(), ['Annee', 'Kilometrage'])
#     ])

# # Complete pipeline with the regressor
# model_pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('xgb', XGBRegressor())
# ])


In [190]:
features = data[['Marque', "Modele" ,'Transmission', 'Carburant', 'Annee', 'Kilometrage']]
target = data['Prix']

le = LabelEncoder()
# Preprocessing pipeline for categorical and numerical data
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Marque', 'Transmission', 'Carburant']),
        ('num', StandardScaler(), ['Annee', 'Kilometrage'])
    ])


# Create a custom transformer function for LabelEncoder
def label_encode(X):
  return le.fit_transform(X)

# Add the custom transformer to the pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('label_encode', Pipeline([('encode', label_encode)]), ["Modele"]),  # Apply to "Modele" column only
    ('xgb', XGBRegressor())
])
# Complete pipeline with the regressor
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('xgb', XGBRegressor())
])


In [191]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=42)

# Train the model
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Marque', 'Transmission',
                                                   'Carburant']),
                                                 ('num', StandardScaler(),
                                                  ['Annee', 'Kilometrage'])])),
                ('xgb',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [192]:
y_pred = model_pipeline.predict(X_test)

In [193]:
mse = mean_squared_error(y_test, y_pred)

In [194]:
np.sqrt(mse)

19305.12074006742

In [195]:
r2_score(y_test,y_pred)

0.23008546464697976

In [196]:
dump(model_pipeline, 'car_price_model_pipeline.joblib')

['car_price_model_pipeline.joblib']

In [197]:
X_train

,Marque,Modele,Transmission,Carburant,Annee,Kilometrage
49,Volkswagen,Golf,Manuelle,Essence,2014,45000
107,Renault,Megane,Manuelle,Diesel,2007,192983
59,Peugeot,208,Manuelle,Diesel,2014,135000
507,BMW,X1,Automatique,Essence,2012,42000
318,Fiat,Punto Evo,Manuelle,Essence,2011,80000
...,...,...,...,...,...,...
121,Fiat,500,Manuelle,Essence,2013,25000
1044,Fiat,Palio,Manuelle,Essence,2010,74000
1095,Renault,Symbol,Manuelle,Essence,2010,35000
860,Peugeot,206,Manuelle,Essence,2001,190000
